It looks like the data was in pretty good shape from your last file 'data_cleaning.ipynb.' Now we should move on to answering questions the marketing team has for you in your interview: Customer Segments, Product Information, and Marketing Return on Investment.

Customer Segment Questions:
1. Segment customers based on demographic data (age, gender, location)
2. Which demographics represent High Frequency Customers? 
3. Which demographics represent Low Frequency Customers?
4. HF vs. LF customers more likely to use a discount code?
5. Average Order Value from High Frequency Customers, AOV for Low Frequency Customers
6. What is the Customer Lifetime Value based on 2022 purchasing data?

Product Information Questions:
1. Which categories have the highest conversion rate? Which products have the highest conversion rate?
2. Top seasonality by total sales 
3. Total sales amount by month and marketing campaign (Which marketing campaigns had the highest total sales?)

Marketing Return on Investment:
1. Compare CPAs for Tik Tok vs. Email 
2. Which marketing campaigns had the highest conversions?
3. Which demographic had the highest conversion rate?

In [61]:
import pandas as pd
import numpy as np
import random
import sqlite3
import sqlalchemy

You start exploring their customer base via customer demographics - a good place to start would be age, gender and location. 

In [62]:
conn = sqlite3.connect('cl_shopper_trends.db')
cursor = conn.cursor()

customer_segment_1 = '''
    SELECT
    Location,
    CASE
    WHEN Age < 30 AND Gender = 'Female' THEN 'Young Female'
    WHEN Age < 30 AND Gender = 'Male' THEN 'Young Male'
    WHEN Age >= 30 AND Gender = 'Female' THEN 'Adult Female'
    WHEN Age >= 30 AND Gender = 'Male' THEN 'Adult Male'
    ELSE 'Other'
    END AS Customer_Segment,
    COUNT(*) AS Segment_Count
    FROM merged_data
    GROUP BY Location, Customer_Segment
    ORDER BY Segment_Count DESC;
    '''
customer_segment_1 = pd.read_sql(customer_segment_1, conn)

# Display the dataframe
print(customer_segment_1)
conn.close()

        Location Customer_Segment  Segment_Count
0       Delaware       Adult Male             25
1         Nevada       Adult Male             23
2      Minnesota       Adult Male             22
3       Missouri       Adult Male             22
4       Kentucky       Adult Male             21
..           ...              ...            ...
95    New Jersey       Young Male              2
96  Rhode Island       Young Male              2
97       Vermont       Young Male              2
98     Wisconsin       Young Male              2
99        Kansas       Young Male              1

[100 rows x 3 columns]


Great, so far it looks like our top customer segments for 2022 were Adult Males. 
By using data from mock_data, let's see if we can categorize our customer segments into High Frequency and Low Frequency segments. 

In [63]:
conn = sqlite3.connect('cl_shopper_trends.db')
cursor = conn.cursor()

customer_segment_2 = '''
SELECT
    Location,
    CASE
        WHEN Age < 30 AND Gender = 'Female' THEN 'Young Female'
        WHEN Age < 30 AND Gender = 'Male' THEN 'Young Male'
        WHEN Age >= 30 AND Gender = 'Female' THEN 'Adult Female'
        WHEN Age >= 30 AND Gender = 'Male' THEN 'Adult Male'
        ELSE 'Other'
    END AS Customer_Segment,
    COUNT(*) AS Segment_Count
FROM merged_data
WHERE Frequency_of_Purchases IN ("Weekly", "Biweekly", "Monthly") 
GROUP BY Location, Customer_Segment
ORDER BY Segment_Count DESC;
'''
customer_segment_2 = pd.read_sql(customer_segment_2, conn)

# Display the dataframe
print(customer_segment_2)
conn.close()

         Location Customer_Segment  Segment_Count
0          Nevada       Adult Male             11
1        Delaware       Adult Male             10
2            Iowa       Adult Male              9
3   Massachusetts       Adult Male              9
4           Idaho       Adult Male              8
..            ...              ...            ...
84   South Dakota       Young Male              1
85      Tennessee       Young Male              1
86           Utah       Young Male              1
87  West Virginia       Young Male              1
88        Wyoming       Young Male              1

[89 rows x 3 columns]


In [64]:
conn = sqlite3.connect('cl_shopper_trends.db')
cursor = conn.cursor()

customer_segment_3 = '''
SELECT
    Location,
    CASE
        WHEN Age < 30 AND Gender = 'Female' THEN 'Young Female'
        WHEN Age < 30 AND Gender = 'Male' THEN 'Young Male'
        WHEN Age >= 30 AND Gender = 'Female' THEN 'Adult Female'
        WHEN Age >= 30 AND Gender = 'Male' THEN 'Adult Male'
        ELSE 'Other'
    END AS Customer_Segment,
    COUNT(*) AS Segment_Count
FROM merged_data
WHERE Frequency_of_Purchases IN ("Quarterly","Annually") 
GROUP BY Location, Customer_Segment
ORDER BY Segment_Count DESC;
'''
customer_segment_3 = pd.read_sql(customer_segment_3, conn)

# Display the dataframe
print(customer_segment_3)
conn.close()

          Location Customer_Segment  Segment_Count
0         Missouri       Adult Male             10
1         Oklahoma       Adult Male              9
2   South Carolina       Adult Male              9
3         Colorado       Adult Male              8
4          Florida       Adult Male              8
..             ...              ...            ...
79            Ohio       Young Male              1
80    Pennsylvania       Young Male              1
81  South Carolina       Young Male              1
82       Tennessee       Young Male              1
83      Washington       Young Male              1

[84 rows x 3 columns]


In [68]:
#Customer Segment #4:
#Are High Frequency Customers or Low Frequency Customers more likely to use a discount code?
#If High Frequency then Frequency_of_Purchases = weekly, biweekly, monthly;  if Low Frequency then Frequency_of_Purchases = quarterly, yearly;SELECT Frequency_of_Purchases, Discount_Applied

conn = sqlite3.connect('cl_shopper_trends.db')
cursor = conn.cursor()

customer_segment_4 = '''
SELECT
Frequency_of_Purchases,
Discount_Applied,
COUNT(Frequency_of_Purchases) AS Frequency_Segment
FROM
merged_data
GROUP BY
Frequency_of_Purchases, Discount_Applied
ORDER BY
Frequency_Segment DESC;

'''

customer_segment_4 = pd.read_sql(customer_segment_4, conn)

# Display the dataframe
print(customer_segment_4)
conn.close()

  Frequency_of_Purchases Discount_Applied  Frequency_Segment
0         Every 3 Months              Yes                151
1                 Weekly              Yes                151
2               Annually              Yes                148
3                Monthly              Yes                145
4            Fortnightly              Yes                142
5              Quarterly              Yes                132
6              Bi-Weekly              Yes                131


Great, it looks like the Frequency_Segment group Bi-Weekly, Quarterly, and Weekly are the groups most likely to use a discount code applied. 

In [66]:
#Customer Segment #5: Average Order Value from High Frequency Customers 
conn = sqlite3.connect('cl_shopper_trends.db')
cursor = conn.cursor()

customer_segment_5 = '''
SELECT Frequency_of_Purchases, AVG (Purchase_Amount_USD)
FROM merged_data
GROUP BY Frequency_of_Purchases;
'''

customer_segment_5 = pd.read_sql(customer_segment_5, conn)
print(customer_segment_5)
conn.close()

  Frequency_of_Purchases  AVG (Purchase_Amount_USD)
0               Annually                  58.662162
1              Bi-Weekly                  59.633588
2         Every 3 Months                  61.092715
3            Fortnightly                  58.612676
4                Monthly                  59.034483
5              Quarterly                  61.371212
6                 Weekly                  59.271523


In [96]:
#Customer Segment #6. What is the Customer Lifetime Value based on 2022 purchasing data? (See SQL DB LITE)
#In order to calcualte CLV, you first need to calculate Customer Value:
#Calculate Average Order Value 
#Calculate Average Number of Purchases 

conn = sqlite3.connect('cl_shopper_trends.db')
cursor = conn.cursor()

customer_segment_6 = '''
SELECT AVG(Purchase_Amount_USD) * AVG(Previous_Purchases) as Customer_Value
FROM merged_data;
'''

'''
SELECT SUM (Previous_Purchases) / SUM (Customer_ID)
FROM merged data;

'''



customer_segment_6 = pd.read_sql(customer_segment_6, conn)
print(customer_segment_6)

#Calculate Customer Lifetime Value (CLV):
#CLV = Customer Value * Avg Customer Lifespan 

   Customer_Value
0     1562.848947


In [69]:
#Product Information #1:  Which products had the highest conversion rate?

conn = sqlite3.connect('cl_shopper_trends.db')
cursor = conn.cursor()

product_information_1 = '''
SELECT Item_Purchased, tiktok_campaign, COUNT (tiktok_conversion), email_campaign, COUNT (email_conversion)
FROM merged_data
GROUP BY tiktok_campaign, email_campaign
ORDER BY tiktok_conversion DESC, email_conversion DESC;
'''

product_information_1 = pd.read_sql(product_information_1, conn)
print(product_information_1)

   Item_Purchased     tiktok_campaign  COUNT (tiktok_conversion)  \
0         Sandals       we missed you                         16   
1          Hoodie      product launch                         31   
2            Coat       cross product                         49   
3         Sweater      abandoned cart                         20   
4         T-shirt      abandoned cart                          6   
5        Backpack      product launch                          8   
6             Hat      product launch                         25   
7           Pants  frequency purchase                         24   
8           Shirt       cross product                        104   
9          Jacket      abandoned cart                          2   
10          Socks             welcome                         46   
11           Coat             welcome                          6   
12          Pants             welcome                         13   
13            Hat       we missed you           

In [70]:
#Product Information #2 Total Sales Amount for seasonal clothing

conn = sqlite3.connect('cl_shopper_trends.db')
cursor = conn.cursor()

product_information_2 = '''
SELECT Season, Total (Purchase_Amount_USD) DESC
FROM merged_data
GROUP BY Season;
'''

product_information_2 = pd.read_sql(product_information_2, conn)
print(product_information_2)

   Season     DESC
0    Fall  15430.0
1  Spring  15220.0
2  Summer  14357.0
3  Winter  14646.0


In [71]:
#Product Information #3: Total sales amount by month and marketing campaign
#Find function for month 

conn = sqlite3.connect('cl_shopper_trends.db')
cursor = conn.cursor()

product_information_3 = '''
SELECT
    email_campaign,
    email_dates,
    tiktok_campaign,
    tiktok_dates,
    SUM(Purchase_Amount_USD) AS Total_Purchase_Amount
FROM
    merged_data
GROUP BY
    email_campaign, email_dates, tiktok_campaign, tiktok_dates;
'''

product_information_3 = pd.read_sql(product_information_3, conn)
print(product_information_3)

     email_campaign email_dates     tiktok_campaign tiktok_dates  \
0    abandoned cart     1/12/22       cross product       3/5/22   
1    abandoned cart     1/12/22       we missed you      8/12/22   
2    abandoned cart     1/13/22  frequency purchase      3/25/22   
3    abandoned cart     1/14/22  frequency purchase       6/5/22   
4    abandoned cart     1/19/22       cross product      3/21/22   
..              ...         ...                 ...          ...   
995         welcome     9/30/22       cross product      7/18/22   
996         welcome     9/30/22       cross product      8/28/22   
997         welcome     9/30/22       we missed you      10/3/22   
998         welcome      9/6/22  frequency purchase      1/22/22   
999         welcome      9/8/22       we missed you      10/9/22   

     Total_Purchase_Amount  
0                       60  
1                       38  
2                       89  
3                       40  
4                       82  
..       

In [72]:
#Marketing ROI #1: Compare CPAs for Tik Tok vs. Email 
#Remove dollar sign and CAST to FLOAT

conn = sqlite3.connect('cl_shopper_trends.db')
cursor = conn.cursor()

marketing_roi_1 = '''
SELECT
  AVG(CAST(REPLACE(tiktok_cpa, '$', '') AS FLOAT)) AS avg_tiktok_cpa,
  AVG(CAST(REPLACE(email_cpa, '$', '') AS FLOAT)) AS avg_email_cpa
FROM merged_data; 
'''

marketing_roi_1 = pd.read_sql(marketing_roi_1, conn)
print(marketing_roi_1)

   avg_tiktok_cpa  avg_email_cpa
0         4.96912        5.11969


It looks like this e-commerce company had nearly equal CPAs across their two campaigns. In this instance, we want to focus on which campaigns had the highest sales or overall conversios.

In [89]:
#Marketing ROI #2: Which marketing campaigns had the highest conversions?

conn = sqlite3.connect('cl_shopper_trends.db')
cursor = conn.cursor()

marketing_roi_2 = '''
SELECT
email_campaign,
tiktok_campaign,
COUNT(*) AS Conversion_Count
FROM
merged_data
WHERE
email_conversion = TRUE AND tiktok_conversion = TRUE
GROUP BY
email_campaign, tiktok_campaign
ORDER BY
Conversion_Count DESC;
'''

marketing_roi_2 = pd.read_sql(marketing_roi_2, conn)
print(marketing_roi_2)

        email_campaign     tiktok_campaign  Conversion_Count
0              welcome       cross product                28
1              welcome  frequency purchase                28
2   frequency purchase  frequency purchase                19
3       abandoned cart       cross product                15
4       abandoned cart  frequency purchase                12
5        cross product  frequency purchase                10
6              welcome             welcome                10
7       abandoned cart      product launch                 9
8   frequency purchase       cross product                 9
9   frequency purchase      product launch                 9
10             welcome      product launch                 9
11      abandoned cart             welcome                 7
12             welcome      abandoned cart                 7
13       cross product       cross product                 6
14  frequency purchase      abandoned cart                 4
15      product launch  

In [91]:
#Marketing ROI #3: Which demographic had the highest conversion rate?

conn = sqlite3.connect('cl_shopper_trends.db')
cursor = conn.cursor()

marketing_roi_3 = '''
SELECT
    Location,
    CASE
        WHEN Age < 30 AND Gender = 'Female' THEN 'Young Female'
        WHEN Age < 30 AND Gender = 'Male' THEN 'Young Male'
        WHEN Age >= 30 AND Gender = 'Female' THEN 'Adult Female'
        WHEN Age >= 30 AND Gender = 'Male' THEN 'Adult Male'
        ELSE 'Other'
    END AS Customer_Segment,
    COUNT(CASE
            WHEN Age < 30 AND Gender = 'Female' THEN 'Young Female'
            WHEN Age < 30 AND Gender = 'Male' THEN 'Young Male'
            WHEN Age >= 30 AND Gender = 'Female' THEN 'Adult Female'
            WHEN Age >= 30 AND Gender = 'Male' THEN 'Adult Male'
            ELSE 'Other'
          END) AS Segment_Count,
    AVG(tiktok_conversion) AS Avg_TikTok_Conversion,
    AVG(email_conversion) AS Avg_Email_Conversion
FROM
    merged_data
GROUP BY
    Location, Customer_Segment;

'''

marketing_roi_3 = pd.read_sql(marketing_roi_3, conn)
print(marketing_roi_3)

         Location Customer_Segment  Segment_Count  Avg_TikTok_Conversion  \
0         Alabama       Adult Male             16               0.500000   
1         Alabama       Young Male              6               0.333333   
2          Alaska       Adult Male             14               0.285714   
3          Alaska       Young Male              4               0.750000   
4         Arizona       Adult Male             12               0.416667   
..            ...              ...            ...                    ...   
95  West Virginia       Young Male              8               0.625000   
96      Wisconsin       Adult Male             11               0.272727   
97      Wisconsin       Young Male              2               0.000000   
98        Wyoming       Adult Male             15               0.400000   
99        Wyoming       Young Male              5               0.800000   

    Avg_Email_Conversion  
0               0.625000  
1               0.666667  
2     